# Portland Crime

## Import packages and load and clean data

In [1]:
import pandas as pd
import os

In [2]:
path = os.path.join("Resources", "crime_incident_data2017.csv")

In [3]:
df = pd.read_csv(path)\
    .rename(columns=lambda s: s.lower().replace(" ", "_"))\
    [[
    'neighborhood',
    'occur_date',
    'occur_time',
    'offense_category',
    'offense_count',
    'offense_type',
    'report_date'
]]\
    .dropna()\
    .assign(report_date=lambda df: pd.to_datetime(df['report_date'], format="%m/%d/%y"))

occur_datetime = pd.to_datetime(
    df["occur_date"] + ":" + df["occur_time"].astype("str").apply(lambda s: s[:-2] + ":" + s[-2:]),
    format="%m/%d/%y:%H:%M",
    errors="coerce"
)

df = df.assign(occur_datetime=occur_datetime)\
    .drop("occur_date", axis=1)\
    .drop("occur_time", axis=1)\
    .dropna()

In [4]:
df.loc[df['offense_count'] >= 2, "offense_category"].unique()

array(['Motor Vehicle Theft'], dtype=object)

In [5]:
df.dtypes

neighborhood                object
offense_category            object
offense_count                int64
offense_type                object
report_date         datetime64[ns]
occur_datetime      datetime64[ns]
dtype: object

## Analysis

In [6]:
df.head()

,neighborhood,offense_category,offense_count,offense_type,report_date,occur_datetime
1,Centennial,Fraud Offenses,1,Identity Theft,2017-01-20,2000-01-20 16:15:00
2,Montavilla,Fraud Offenses,1,False Pretenses/Swindle/Confidence Game,2017-01-09,2003-12-01 08:00:00
5,King,Fraud Offenses,1,Identity Theft,2017-01-03,2010-11-28 16:12:00
6,Vernon,Fraud Offenses,1,False Pretenses/Swindle/Confidence Game,2017-01-26,2013-11-08 12:00:00
7,Vernon,Fraud Offenses,1,Identity Theft,2017-01-26,2013-11-08 12:00:00


Most Crime Neighborhoods (by incident)

In [7]:
neighborhood_crime_count = df["neighborhood"].value_counts()

In [8]:
neighborhood_crime_count.head(10)

Downtown               2650
Hazelwood              2484
Lents                  1422
Powellhurst-Gilbert    1355
Old Town/Chinatown     1242
Centennial             1181
Northwest              1095
Montavilla              993
Pearl                   972
Lloyd                   920
Name: neighborhood, dtype: int64

In [9]:
neighborhood_crime_count.tail(10).sort_values(ascending=True)

Marshall Park         5
Healy Heights         5
Crestwood             6
Woodland Park        19
Arnold Creek         21
South Burlingame     23
Markham              24
Northwest Heights    25
Hillside             29
Ardenwald            32
Name: neighborhood, dtype: int64

Most Crime Neighborhoods (offense_count)

In [10]:
vehicle_df = df[df["offense_type"]=="Motor Vehicle Theft"]

In [11]:
vehicle_df["neighborhood"].value_counts()

Hazelwood              329
Lents                  240
Powellhurst-Gilbert    194
Centennial             187
Montavilla             157
                      ... 
Far Southwest            1
Crestwood                1
Northwest Heights        1
Maplewood                1
Woodland Park            1
Name: neighborhood, Length: 90, dtype: int64

In [12]:
vehicle_df.groupby("neighborhood")["offense_count"].sum().sort_values(ascending=False)

neighborhood
Hazelwood              331
Lents                  240
Powellhurst-Gilbert    197
Centennial             188
Montavilla             158
                      ... 
Maplewood                1
Northwest Heights        1
Woodland Park            1
Crestwood                1
Far Southwest            1
Name: offense_count, Length: 90, dtype: int64

Most Crime Hour (by offense_count)

In [13]:
df.groupby(pd.Grouper(key="occur_datetime", freq="5B"))["offense_count"].sum()

occur_datetime
1979-09-14       1
1979-09-21       0
1979-09-28       0
1979-10-05       0
1979-10-12       0
              ... 
2017-07-28    1042
2017-08-04    1043
2017-08-11    1003
2017-08-18    1015
2017-08-25     954
Freq: 5B, Name: offense_count, Length: 1981, dtype: int64

In [14]:
df.groupby(df["occur_datetime"].dt.hour)["offense_count"].sum()

occur_datetime
1     1300
2     1123
3      909
4      772
5      717
6      781
7     1089
8     1578
9     1439
10    1450
11    1473
12    2366
13    1622
14    1676
15    1855
16    1933
17    2267
18    2259
19    2114
20    2126
21    2106
22    2328
23    1947
Name: offense_count, dtype: int64